<a href="https://colab.research.google.com/github/user-1221/home-pa-algo/blob/main/homepa_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Home-PA Daily Allocator (Python)

Run this notebook end-to-end in Google Colab to reproduce the scheduling behaviour from the TypeScript implementation. Cells below load sample data (20+ memos), run the allocator, print the debug trace, and render simple visualisations so you can inspect how suggestions compete for open time blocks.


In [ ]:
import math
import random
from dataclasses import dataclass, field, asdict
from datetime import datetime, timedelta, time, timezone
from typing import List, Optional, Dict, Any, Tuple, Callable

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from dateutil import parser



In [ ]:
DEFAULT_SEED = 20251107
DEFAULT_DELTA = 5
DEFAULT_TRAVEL_SPEED = 40  # km/h
DEFAULT_SAFETY_BUFFER = 10
EARTH_RADIUS_KM = 6371


def clamp(value: float, min_value: float, max_value: float) -> float:
    return max(min_value, min(max_value, value))


def _imul(a: int, b: int) -> int:
    return ((a & 0xFFFFFFFF) * (b & 0xFFFFFFFF)) & 0xFFFFFFFF


def _u32(value: int) -> int:
    return value & 0xFFFFFFFF


def _rshift(value: int, bits: int) -> int:
    return (value & 0xFFFFFFFF) >> bits


def create_seeded_rng(seed: int) -> Callable[[], float]:
    state = seed & 0xFFFFFFFF

    def rng() -> float:
        nonlocal state
        state = (state + 0x6D2B79F5) & 0xFFFFFFFF
        t = _imul(state ^ _rshift(state, 15), 1 | state)
        imul_part = _imul(t ^ _rshift(t, 7), 61 | t)
        t = (t ^ ((_u32(t) + imul_part) & 0xFFFFFFFF)) & 0xFFFFFFFF
        result = (t ^ _rshift(t, 14)) & 0xFFFFFFFF
        return result / 4294967296

    return rng


def random_in_range(rng: Callable[[], float], min_value: float, max_value: float) -> float:
    return min_value + (max_value - min_value) * rng()


def random_int_in_range(rng: Callable[[], float], min_value: int, max_value: int) -> int:
    return int(random_in_range(rng, min_value, max_value + 1))


def parse_iso(iso_str: str) -> datetime:
    return parser.isoparse(iso_str)


def format_iso(dt: datetime) -> str:
    if dt.tzinfo is None:
        return dt.replace(tzinfo=timezone.utc).isoformat().replace("+00:00", "Z")
    return dt.astimezone(timezone.utc).isoformat().replace("+00:00", "Z")


def add_minutes(dt: datetime, minutes: float) -> datetime:
    return dt + timedelta(minutes=minutes)


def difference_in_minutes(start: datetime, end: datetime) -> float:
    return (end - start).total_seconds() / 60


def get_start_of_day(dt: datetime) -> datetime:
    return dt.replace(hour=0, minute=0, second=0, microsecond=0)


def get_end_of_day(dt: datetime) -> datetime:
    return dt.replace(hour=23, minute=59, second=59, microsecond=999000)


def minutes_since_start_of_day(dt: datetime) -> float:
    return dt.hour * 60 + dt.minute


def haversine_distance_km(a: Optional[Dict[str, float]], b: Optional[Dict[str, float]]) -> float:
    if not a or not b:
        return 0.0
    lat1, lng1 = a["lat"], a["lng"]
    lat2, lng2 = b["lat"], b["lng"]
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    d_phi = math.radians(lat2 - lat1)
    d_lambda = math.radians(lng2 - lng1)
    h = math.sin(d_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(d_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(h), math.sqrt(1 - h))
    return EARTH_RADIUS_KM * c


def travel_minutes(distance_km: float, speed_kmh: float) -> float:
    if speed_kmh <= 0:
        return math.inf
    return (distance_km / speed_kmh) * 60


def hash_to_seed(text: str) -> int:
    hash_value = 0
    for ch in text:
        hash_value = (_imul(31, hash_value) + ord(ch)) & 0xFFFFFFFF
    return hash_value & 0xFFFFFFFF



In [ ]:
@dataclass
class TimeWindow:
    start: str
    end: str
    confidence: Optional[float] = None


@dataclass
class CalendarEvent:
    id: str
    start_iso: str
    end_iso: str
    location: Optional[Dict[str, float]] = None
    title: Optional[str] = None
    locked: Optional[bool] = None


@dataclass
class Memo:
    id: str
    title: str
    type: str
    importance: float
    deadline: Optional[str] = None
    frequency: Optional[str] = None
    last_done: Optional[str] = None
    ideal_time_windows: Optional[List[TimeWindow]] = None
    place_type: Optional[str] = None
    location: Optional[Dict[str, float]] = None
    explicit_duration_min: Optional[int] = None
    interruptible: Optional[bool] = True
    must_not_split: Optional[bool] = False


@dataclass
class WeightParameters:
    alpha: float
    beta: float
    gamma: float


@dataclass
class AllocationOptions:
    seed: Optional[int] = None
    deltaMinutes: Optional[int] = None
    travelSpeedKmh: Optional[float] = None
    safetyBufferMinutes: Optional[int] = None
    weightOverrides: Optional[Dict[str, float]] = None


@dataclass
class AllocationParameters:
    seed: int
    weights: WeightParameters
    deltaMinutes: int
    travelSpeedKmh: float
    safetyBufferMinutes: int
    tPrefByPlace: Dict[str, float]
    contPenalty: Dict[str, float]
    switchCost: Dict[str, float]


@dataclass
class ScheduledItemReason:
    score: float
    need: float
    obstacle: float
    fit: float


@dataclass
class ScheduledItem:
    id: str
    memo_id: str
    start_iso: str
    end_iso: str
    assigned_duration_min: float
    reason: ScheduledItemReason


@dataclass
class DebugLogEntry:
    timestamp_iso: str
    memo_id: str
    delta_min: float
    delta_gain: float
    L_before: float
    L_after: float


@dataclass
class AllocationResult:
    scheduled_items: List[ScheduledItem]
    debug_log: List[DebugLogEntry]
    infeasible: List[str]
    parameters: AllocationParameters


@dataclass
class Gap:
    start: datetime
    end: datetime
    prevLocation: Optional[Dict[str, float]] = None
    nextLocation: Optional[Dict[str, float]] = None

    @property
    def length_minutes(self) -> float:
        return difference_in_minutes(self.start, self.end)



In [ ]:
@dataclass
class MemoState:
    memo: Memo
    duration: float
    remaining: float
    total_assigned: float = 0
    blocks: List[Dict[str, Any]] = field(default_factory=list)
    current_block_length: float = 0
    need: float = 0
    est_confidence: float = 0
    t_pref: float = 60
    assigned: bool = False


@dataclass
class GapState:
    gap: Gap
    used_minutes: float = 0
    last_assigned_end: Optional[datetime] = None
    active_memo_id: Optional[str] = None
    last_memo_location: Optional[Dict[str, float]] = None


@dataclass
class NormalizedEvent:
    raw: CalendarEvent
    start: datetime
    end: datetime
    location: Optional[Dict[str, float]]


@dataclass
class GainContext:
    memo_state: MemoState
    gap_state: GapState
    available_minutes: float
    current_start: datetime
    travel_prev: float
    travel_next: float
    gap_length: float
    remaining_for_memo: float
    additional_buffer: float


def build_parameters(options: AllocationOptions, seed: int) -> AllocationParameters:
    rng = create_seeded_rng(seed & 0xFFFFFFFF)
    weights = WeightParameters(
        alpha=options.weightOverrides.get("alpha", 1.0) if options.weightOverrides else 1.0,
        beta=options.weightOverrides.get("beta", 0.8) if options.weightOverrides else 0.8,
        gamma=options.weightOverrides.get("gamma", 0.5) if options.weightOverrides else 0.5,
    )

    t_pref_by_place: Dict[str, float] = {}

    cont_penalty = {
        "p0": random_in_range(rng, 0.005, 0.02),
        "alpha": random_in_range(rng, 1.1, 1.6),
        "tScale": random_in_range(rng, 20, 45),
    }
    switch_cost = {
        "base": random_in_range(rng, 0.3, 1.0),
        "lambdaLocation": random_in_range(rng, 0.2, 0.6),
        "tau": random_in_range(rng, 20, 45),
    }

    return AllocationParameters(
        seed=seed,
        weights=weights,
        deltaMinutes=options.deltaMinutes or DEFAULT_DELTA,
        travelSpeedKmh=options.travelSpeedKmh or DEFAULT_TRAVEL_SPEED,
        safetyBufferMinutes=options.safetyBufferMinutes or DEFAULT_SAFETY_BUFFER,
        tPrefByPlace=t_pref_by_place,
        contPenalty=cont_penalty,
        switchCost=switch_cost,
    )


def normalize_events(events: List[CalendarEvent]) -> List[NormalizedEvent]:
    normalized: List[NormalizedEvent] = []
    for event in events:
        start = parse_iso(event.start_iso)
        end = parse_iso(event.end_iso)
        normalized.append(NormalizedEvent(raw=event, start=start, end=end, location=event.location))
    normalized.sort(key=lambda evt: evt.start)
    return normalized


def derive_day_bounds(events: List[NormalizedEvent], memos: List[Memo]) -> Tuple[datetime, datetime]:
    now = datetime.now(timezone.utc)
    if not events:
        day_start = get_start_of_day(now)
        return day_start, get_end_of_day(day_start)

    earliest_event = events[0].start
    day_base = get_start_of_day(earliest_event)

    min_window_minutes: Optional[float] = None
    for memo in memos:
        if not memo.ideal_time_windows:
            continue
        for window in memo.ideal_time_windows:
            try:
                hour, minute = map(int, window.start.split(":"))
            except ValueError:
                continue
            total_minutes = hour * 60 + minute
            if min_window_minutes is None or total_minutes < min_window_minutes:
                min_window_minutes = total_minutes

    earliest_event_minutes = earliest_event.hour * 60 + earliest_event.minute
    default_lead = max(0, earliest_event_minutes - 120)
    candidates = [earliest_event_minutes]
    if min_window_minutes is not None:
        candidates.append(min_window_minutes)
    if default_lead != 0:
        candidates.append(default_lead)
    start_minutes = min(candidates)

    day_start = day_base.replace(hour=int(start_minutes // 60), minute=int(start_minutes % 60), second=0, microsecond=0)
    day_end = get_end_of_day(events[-1].end)
    return day_start, day_end


def build_gaps(events: List[NormalizedEvent], day_start: datetime, day_end: datetime) -> List[Gap]:
    gaps: List[Gap] = []
    cursor = day_start
    prev_location: Optional[Dict[str, float]] = None

    for event in events:
        if event.start > cursor:
            gaps.append(
                Gap(
                    start=cursor,
                    end=event.start,
                    prevLocation=prev_location,
                    nextLocation=event.location,
                )
            )
        if event.end > cursor:
            cursor = event.end
        prev_location = event.location

    if cursor < day_end:
        gaps.append(Gap(start=cursor, end=day_end, prevLocation=prev_location, nextLocation=None))

    return [gap for gap in gaps if difference_in_minutes(gap.start, gap.end) > 0]


def estimate_duration(memo: Memo) -> Tuple[float, float]:
    if memo.explicit_duration_min and memo.explicit_duration_min > 0:
        return float(memo.explicit_duration_min), 1.0

    mapping: Dict[str, float] = {
        "meeting": 30,
        "study": 60,
        "quick": 15,
        "exercise": 45,
    }
    key = memo.place_type or memo.type
    duration = mapping.get(key, 30)
    return float(duration), 0.5


def compute_need(memo: Memo, day_start: datetime) -> float:
    def deadline_urgency() -> float:
        if not memo.deadline:
            return 0.0
        deadline = parse_iso(memo.deadline)
        hours = (deadline - day_start).total_seconds() / 3600
        t0 = 48
        tau = 12
        return clamp(1 / (1 + math.exp((hours - t0) / tau)), 0, 1)

    def frequency_urgency() -> float:
        if not memo.frequency or memo.frequency == "one-off":
            return 0.0
        periods = {
            "daily": 24,
            "weekly": 24 * 7,
            "monthly": 24 * 30,
        }
        period_hours = periods.get(memo.frequency, 24 * 7)
        if not memo.last_done:
            return 1.0
        last = parse_iso(memo.last_done)
        elapsed = (day_start - last).total_seconds() / 3600
        if elapsed <= 0:
            return 0.0
        return clamp(elapsed / period_hours, 0, 1)

    importance = clamp(memo.importance, 0, 1)
    value = 0.5 * deadline_urgency() + 0.25 * frequency_urgency() + 0.25 * importance
    return clamp(value, 0, 1)


def get_t_pref_for_memo(memo: Memo, params: AllocationParameters, fallback: float) -> float:
    if not memo.place_type:
        return max(fallback, 60)
    if memo.place_type not in params.tPrefByPlace:
        seed = hash_to_seed(f"{memo.place_type}:{memo.id}:{params.seed}")
        rng = create_seeded_rng(seed)
        params.tPrefByPlace[memo.place_type] = round(random_in_range(rng, 60, 240))
    return params.tPrefByPlace[memo.place_type]


def initialize_memo_states(memos: List[Memo], day_start: datetime, params: AllocationParameters) -> List[MemoState]:
    states: List[MemoState] = []
    for memo in memos:
        duration, confidence = estimate_duration(memo)
        need = compute_need(memo, day_start)
        t_pref = get_t_pref_for_memo(memo, params, duration)
        states.append(
            MemoState(
                memo=memo,
                duration=duration,
                remaining=duration,
                total_assigned=0,
                blocks=[],
                current_block_length=0,
                need=need,
                est_confidence=confidence,
                t_pref=t_pref,
                assigned=False,
            )
        )
    return states


def compute_travel_minutes(a: Optional[Dict[str, float]], b: Optional[Dict[str, float]], speed_kmh: float) -> float:
    distance = haversine_distance_km(a, b)
    if not math.isfinite(distance):
        return 0.0
    return travel_minutes(distance, speed_kmh)


def locations_differ(a: Optional[Dict[str, float]], b: Optional[Dict[str, float]]) -> bool:
    if not a and not b:
        return False
    if not a or not b:
        return True
    return abs(a["lat"] - b["lat"]) > 0.0001 or abs(a["lng"] - b["lng"]) > 0.0001


def compute_timing_mismatch(memo: Memo, slot_start: datetime, delta: float) -> float:
    if not memo.ideal_time_windows:
        return 0.5
    start_minutes = minutes_since_start_of_day(slot_start)
    end_minutes = start_minutes + delta
    for window in memo.ideal_time_windows:
        try:
            wh, wm = map(int, window.start.split(":"))
            eh, em = map(int, window.end.split(":"))
        except ValueError:
            continue
        w_start = wh * 60 + wm
        w_end = eh * 60 + em
        if end_minutes <= w_start or start_minutes >= w_end:
            continue
        overlap = min(end_minutes, w_end) - max(start_minutes, w_start)
        if overlap > 0:
            ratio = overlap / delta if delta > 0 else 0
            return clamp(1 - ratio, 0, 1)
    return 1.0


def marginal_utility(state: MemoState, delta: float) -> float:
    k = state.t_pref or 60
    u_max = 5 * state.need
    current = state.current_block_length
    return (u_max / (k + 1)) * math.exp(-current / (k + 1)) * delta


def continuity_penalty(state: MemoState, delta: float, params: AllocationParameters) -> float:
    L = state.current_block_length + delta
    over = max(0, (L - state.t_pref) / max(1, params.contPenalty["tScale"]))
    return params.contPenalty["p0"] * (over ** params.contPenalty["alpha"]) * delta


def compute_switch_cost(state: MemoState, context: GainContext, params: AllocationParameters) -> float:
    gap_state = context.gap_state
    current_start = context.current_start
    if not gap_state.last_assigned_end or gap_state.active_memo_id == state.memo.id:
        return 0.0
    break_len = max(0, difference_in_minutes(gap_state.last_assigned_end, current_start))
    location_changed = 1 if locations_differ(gap_state.last_memo_location, state.memo.location) else 0
    base = params.switchCost["base"]
    lambda_loc = params.switchCost["lambdaLocation"]
    tau = max(1, params.switchCost["tau"])
    return base * (1 + lambda_loc * location_changed) * math.exp(-break_len / tau)


def compute_reason(
    memo_state: MemoState,
    gap_state: GapState,
    delta: float,
    travel_prev: float,
    travel_next: float,
    gap_length: float,
    params: AllocationParameters,
    offset_minutes: float,
) -> Dict[str, float]:
    need = memo_state.need
    travel_component = clamp((travel_prev + travel_next) / max(1, gap_length), 0, 1)
    slot_time = add_minutes(gap_state.gap.start, offset_minutes)
    timing_mismatch = compute_timing_mismatch(memo_state.memo, slot_time, delta)
    back_to_back = 0
    if gap_state.last_assigned_end:
        back_to_back = 1 if difference_in_minutes(gap_state.last_assigned_end, slot_time) < params.safetyBufferMinutes else 0
    obstacle = clamp(0.4 * travel_component + 0.4 * timing_mismatch + 0.2 * back_to_back, 0, 1)

    gap_remaining = max(0, gap_length - offset_minutes)
    target_duration = memo_state.duration
    dur_fit_raw = -1 if gap_length <= 0 else 1 - abs(gap_remaining - target_duration) / gap_length
    dur_fit = clamp(dur_fit_raw, -1, 1)
    place_score = 1 if memo_state.memo.place_type else 0.5
    interrupt_score = (
        1 if memo_state.memo.interruptible is not False else (1 if delta >= target_duration else 0.2)
    )
    window_fit = 1 - timing_mismatch
    fit = clamp((dur_fit + place_score + interrupt_score + window_fit) / 4, -1, 1)

    score = (
        params.weights.alpha * need
        - params.weights.beta * obstacle
        + params.weights.gamma * fit
    )

    return {
        "score": score,
        "need": need,
        "obstacle": obstacle,
        "fit": fit,
    }


def evaluate_delta(delta: float, context: GainContext, params: AllocationParameters) -> Tuple[float, Dict[str, float]]:
    memo_state = context.memo_state
    gap_state = context.gap_state
    reason = compute_reason(
        memo_state,
        gap_state,
        delta,
        context.travel_prev,
        context.travel_next,
        context.gap_length,
        params,
        difference_in_minutes(gap_state.gap.start, context.current_start),
    )
    mu = marginal_utility(memo_state, delta)
    penalty = continuity_penalty(memo_state, delta, params)
    switch_cost = compute_switch_cost(memo_state, context, params)
    travel_cost = clamp((context.travel_prev + context.travel_next) / 120, 0, 1)
    remaining_ratio = 0 if context.remaining_for_memo <= 0 else delta / context.remaining_for_memo
    gain = reason["score"] + mu - penalty - switch_cost - travel_cost - 0.05 * remaining_ratio
    return gain, reason


def update_memo_state_with_allocation(state: MemoState, start: datetime, end: datetime, reason: Dict[str, float]) -> None:
    if state.blocks:
        last_block = state.blocks[-1]
        if last_block["end"] == start:
            last_block["end"] = end
            last_block["reason"] = reason
            return
    state.blocks.append({"start": start, "end": end, "reason": reason})


def finalize_active_memo(gap_state: GapState, memo_states: List[MemoState]) -> None:
    if not gap_state.active_memo_id:
        return
    for state in memo_states:
        if state.memo.id == gap_state.active_memo_id:
            state.current_block_length = 0
            break
    gap_state.active_memo_id = None


def allocate_non_splittable_blocks(
    states: List[MemoState],
    gap_states: List[GapState],
    params: AllocationParameters,
    debug_log: List[Dict[str, Any]],
) -> None:
    sorted_states = sorted(states, key=lambda s: s.need, reverse=True)
    for state in sorted_states:
        if state.duration <= 0:
            continue
        best_gap: Optional[GapState] = None
        best_score = -math.inf
        best_reason: Optional[Dict[str, float]] = None

        for gap_state in gap_states:
            gap = gap_state.gap
            gap_length = difference_in_minutes(gap.start, gap.end)
            travel_prev = compute_travel_minutes(gap.prevLocation, state.memo.location, params.travelSpeedKmh)
            travel_next = compute_travel_minutes(state.memo.location, gap.nextLocation, params.travelSpeedKmh)
            available = gap_length - gap_state.used_minutes - params.safetyBufferMinutes
            if available < state.duration + travel_prev + travel_next:
                continue

            reason = compute_reason(
                state,
                gap_state,
                state.duration,
                travel_prev,
                travel_next,
                gap_length,
                params,
                gap_state.used_minutes,
            )
            score = reason["score"]
            if score > best_score:
                best_score = score
                best_gap = gap_state
                best_reason = reason

        if not best_gap or not best_reason:
            continue

        start = add_minutes(best_gap.gap.start, best_gap.used_minutes)
        end = add_minutes(start, state.duration)

        best_gap.used_minutes += state.duration
        best_gap.last_assigned_end = end
        best_gap.active_memo_id = None
        best_gap.last_memo_location = state.memo.location

        state.blocks.append({"start": start, "end": end, "reason": best_reason})
        state.remaining = 0
        state.total_assigned = state.duration
        state.assigned = True
        state.current_block_length = 0

        t = 0
        while t < state.duration:
            chunk = min(params.deltaMinutes, state.duration - t)
            debug_log.append(
                {
                    "timestamp_iso": format_iso(add_minutes(start, t)),
                    "memo_id": state.memo.id,
                    "delta_min": chunk,
                    "delta_gain": round(best_reason["score"], 3),
                    "L_before": round(t, 2),
                    "L_after": round(t + chunk, 2),
                }
            )
            t += params.deltaMinutes



In [ ]:
def load_sample_day() -> Tuple[List[CalendarEvent], List[Memo]]:
    events = [
        CalendarEvent(
            id="event-morning-meeting",
            start_iso="2025-11-07T09:00:00+09:00",
            end_iso="2025-11-07T10:00:00+09:00",
            title="Product sync",
            locked=True,
            location={"lat": 35.681236, "lng": 139.767125},
        ),
        CalendarEvent(
            id="event-lunch",
            start_iso="2025-11-07T12:00:00+09:00",
            end_iso="2025-11-07T13:00:00+09:00",
            title="Lunch with team",
            locked=True,
            location={"lat": 35.668441, "lng": 139.600784},
        ),
        CalendarEvent(
            id="event-evening-sync",
            start_iso="2025-11-07T18:30:00+09:00",
            end_iso="2025-11-07T19:00:00+09:00",
            title="Client call",
            locked=True,
            location={"lat": 35.710063, "lng": 139.8107},
        ),
    ]

    def tw(start: str, end: str, confidence: float = 0.8) -> TimeWindow:
        return TimeWindow(start=start, end=end, confidence=confidence)

    memo_defs = [
        {
            "id": "memo-study",
            "title": "Study at cafe",
            "type": "task",
            "importance": 0.9,
            "place_type": "cafe",
            "location": {"lat": 35.689487, "lng": 139.691711},
            "explicit_duration_min": 60,
            "ideal_time_windows": [tw("08:00", "11:30")],
        },
        {
            "id": "memo-laundry",
            "title": "Laundry",
            "type": "task",
            "importance": 0.4,
            "place_type": "home",
            "explicit_duration_min": 45,
        },
        {
            "id": "memo-groceries",
            "title": "Grocery run",
            "type": "personal",
            "importance": 0.6,
            "place_type": "store",
            "location": {"lat": 35.6702, "lng": 139.702},
            "explicit_duration_min": 40,
            "must_not_split": True,
        },
        {
            "id": "memo-gym",
            "title": "Gym session",
            "type": "personal",
            "importance": 0.7,
            "place_type": "gym",
            "location": {"lat": 35.658034, "lng": 139.701636},
            "explicit_duration_min": 50,
        },
        {
            "id": "memo-writing",
            "title": "Journal writing",
            "type": "personal",
            "importance": 0.5,
            "place_type": "home",
            "explicit_duration_min": 30,
        },
        {
            "id": "memo-reading",
            "title": "Read research papers",
            "type": "task",
            "importance": 0.8,
            "place_type": "office",
            "location": {"lat": 35.680959, "lng": 139.767306},
            "explicit_duration_min": 45,
            "ideal_time_windows": [tw("08:00", "11:30")],
        },
        {
            "id": "memo-walk",
            "title": "Walk in park",
            "type": "personal",
            "importance": 0.3,
            "place_type": "park",
            "location": {"lat": 35.6698, "lng": 139.7026},
            "explicit_duration_min": 30,
        },
        {
            "id": "memo-email",
            "title": "Inbox zero",
            "type": "task",
            "importance": 0.85,
            "place_type": "office",
            "location": {"lat": 35.680959, "lng": 139.767306},
            "explicit_duration_min": 35,
            "ideal_time_windows": [tw("08:00", "11:30")],
        },
        {
            "id": "memo-planning",
            "title": "Weekly planning",
            "type": "task",
            "importance": 0.65,
            "place_type": "office",
            "location": {"lat": 35.680959, "lng": 139.767306},
            "explicit_duration_min": 25,
        },
        {
            "id": "memo-backlog",
            "title": "Bug triage",
            "type": "task",
            "importance": 0.75,
            "place_type": "office",
            "location": {"lat": 35.680959, "lng": 139.767306},
            "explicit_duration_min": 40,
        },
        {
            "id": "memo-coffee",
            "title": "Coffee with mentor",
            "type": "personal",
            "importance": 0.5,
            "place_type": "cafe",
            "location": {"lat": 35.7003, "lng": 139.772},
            "explicit_duration_min": 30,
            "must_not_split": True,
        },
        {
            "id": "memo-errand-city",
            "title": "City hall errand",
            "type": "personal",
            "importance": 0.55,
            "place_type": "gov",
            "location": {"lat": 35.6938, "lng": 139.7034},
            "explicit_duration_min": 35,
            "must_not_split": True,
        },
        {
            "id": "memo-language",
            "title": "Language practice",
            "type": "task",
            "importance": 0.6,
            "place_type": "home",
            "explicit_duration_min": 45,
        },
        {
            "id": "memo-design",
            "title": "Sketch ideas",
            "type": "personal",
            "importance": 0.45,
            "place_type": "studio",
            "location": {"lat": 35.662, "lng": 139.699},
            "explicit_duration_min": 40,
        },
        {
            "id": "memo-meditate",
            "title": "Meditation",
            "type": "personal",
            "importance": 0.5,
            "place_type": "home",
            "explicit_duration_min": 20,
        },
        {
            "id": "memo-shopping",
            "title": "Online shopping",
            "type": "personal",
            "importance": 0.35,
            "place_type": "home",
            "explicit_duration_min": 25,
        },
        {
            "id": "memo-call-family",
            "title": "Call family",
            "type": "personal",
            "importance": 0.7,
            "place_type": "home",
            "explicit_duration_min": 30,
        },
        {
            "id": "memo-sideproject",
            "title": "Side project coding",
            "type": "task",
            "importance": 0.9,
            "place_type": "home",
            "explicit_duration_min": 90,
        },
        {
            "id": "memo-video",
            "title": "Record vlog",
            "type": "personal",
            "importance": 0.4,
            "place_type": "studio",
            "location": {"lat": 35.662, "lng": 139.699},
            "explicit_duration_min": 45,
        },
        {
            "id": "memo-networking",
            "title": "Reply to community messages",
            "type": "personal",
            "importance": 0.6,
            "place_type": "home",
            "explicit_duration_min": 35,
        },
        {
            "id": "memo-cleaning",
            "title": "Deep clean kitchen",
            "type": "task",
            "importance": 0.5,
            "place_type": "home",
            "explicit_duration_min": 50,
        },
    ]

    memos = [Memo(**definition) for definition in memo_defs]
    assert len(memos) >= 20, "Need at least 20 memos"
    return events, memos



In [ ]:
# Deprecated helper implementations from the initial prototype have been removed.



In [ ]:
# Legacy helper functions intentionally removed.
# The canonical implementations appear earlier in the notebook and accept the
# full AllocationParameters context. This cell is kept empty to avoid
# overriding those definitions.


In [ ]:
# Legacy helper functions removed; see mirrored TypeScript logic above for active implementations.



In [ ]:
def allocate_day(
    calendar_events: List[CalendarEvent],
    memos: List[Memo],
    options: Optional[AllocationOptions] = None,
) -> Dict[str, Any]:
    options = options or AllocationOptions()
    seed = options.seed if options.seed is not None else DEFAULT_SEED
    params = build_parameters(options, seed)

    if not memos:
        return {
            "scheduled_items": [],
            "debug_log": [],
            "infeasible": [],
            "parameters": asdict(params),
        }

    blocked_events = normalize_events(calendar_events)
    day_start, day_end = derive_day_bounds(blocked_events, memos)
    gaps = build_gaps(blocked_events, day_start, day_end)
    memo_states = initialize_memo_states(memos, day_start, params)

    debug_log: List[Dict[str, Any]] = []
    gap_states = [GapState(gap=gap, used_minutes=0, last_assigned_end=None, active_memo_id=None, last_memo_location=gap.prevLocation) for gap in gaps]

    non_split_states = [state for state in memo_states if state.memo.must_not_split]
    if non_split_states:
        allocate_non_splittable_blocks(non_split_states, gap_states, params, debug_log)

    for gap_state in gap_states:
        gap_length = difference_in_minutes(gap_state.gap.start, gap_state.gap.end)
        if gap_state.used_minutes >= gap_length:
            finalize_active_memo(gap_state, memo_states)
            continue

        effective_span = gap_length - params.safetyBufferMinutes
        if effective_span <= 0:
            finalize_active_memo(gap_state, memo_states)
            continue

        while gap_state.used_minutes < gap_length:
            remaining_gap = gap_length - gap_state.used_minutes
            delta = min(params.deltaMinutes, remaining_gap)
            if delta <= 0:
                break

            candidates = [state for state in memo_states if state.remaining > 0 and not state.memo.must_not_split]
            if not candidates:
                break

            best_state: Optional[MemoState] = None
            best_gain = -math.inf
            best_context: Optional[GainContext] = None
            best_reason: Optional[Dict[str, float]] = None
            best_delta = 0.0

            for state in candidates:
                additional_buffer = (
                    params.safetyBufferMinutes
                    if gap_state.active_memo_id and gap_state.active_memo_id != state.memo.id
                    else 0
                )
                available = gap_length - gap_state.used_minutes - additional_buffer
                if available <= 0:
                    continue

                actual_delta = min(delta, state.remaining, available)
                if actual_delta <= 0:
                    continue

                travel_prev = compute_travel_minutes(gap_state.gap.prevLocation, state.memo.location, params.travelSpeedKmh)
                travel_next = compute_travel_minutes(state.memo.location, gap_state.gap.nextLocation, params.travelSpeedKmh)

                if travel_prev + travel_next + actual_delta + params.safetyBufferMinutes > gap_length:
                    continue

                potential_start = add_minutes(gap_state.gap.start, gap_state.used_minutes + additional_buffer)
                context = GainContext(
                    memo_state=state,
                    gap_state=gap_state,
                    available_minutes=available,
                    current_start=potential_start,
                    travel_prev=travel_prev,
                    travel_next=travel_next,
                    gap_length=gap_length,
                    remaining_for_memo=state.remaining,
                    additional_buffer=additional_buffer,
                )
                gain, reason = evaluate_delta(actual_delta, context, params)
                if gain > best_gain:
                    best_gain = gain
                    best_state = state
                    best_context = context
                    best_reason = reason
                    best_delta = actual_delta

            if not best_state or not best_context or not best_reason or best_gain <= 0.01:
                break

            actual_delta = min(best_delta, best_state.remaining)

            if best_context.additional_buffer > 0:
                if gap_state.used_minutes + best_context.additional_buffer >= gap_length:
                    break
                gap_state.used_minutes += best_context.additional_buffer
                finalize_active_memo(gap_state, memo_states)

            slot_start = add_minutes(gap_state.gap.start, gap_state.used_minutes)
            delta_minutes = min(actual_delta, gap_length - gap_state.used_minutes)
            if delta_minutes <= 0:
                break

            slot_end = add_minutes(slot_start, delta_minutes)
            L_before = best_state.current_block_length
            update_memo_state_with_allocation(best_state, slot_start, slot_end, best_reason)
            best_state.remaining = max(0.0, best_state.remaining - delta_minutes)
            best_state.total_assigned += delta_minutes
            best_state.current_block_length += delta_minutes
            best_state.assigned = True

            gap_state.used_minutes += delta_minutes
            gap_state.last_assigned_end = slot_end
            gap_state.active_memo_id = best_state.memo.id
            gap_state.last_memo_location = best_state.memo.location

            debug_log.append(
                {
                    "timestamp_iso": format_iso(slot_start),
                    "memo_id": best_state.memo.id,
                    "delta_min": delta_minutes,
                    "delta_gain": round(best_gain, 3),
                    "L_before": round(L_before, 2),
                    "L_after": round(best_state.current_block_length, 2),
                }
            )

            if best_state.remaining <= 0:
                finalize_active_memo(gap_state, memo_states)

        finalize_active_memo(gap_state, memo_states)

    scheduled_items: List[Dict[str, Any]] = []
    for state in memo_states:
        for idx, block in enumerate(state.blocks, start=1):
            scheduled_items.append(
                {
                    "id": f"{state.memo.id}#{idx}",
                    "memo_id": state.memo.id,
                    "title": state.memo.title,
                    "start_iso": format_iso(block["start"]),
                    "end_iso": format_iso(block["end"]),
                    "assigned_duration_min": difference_in_minutes(block["start"], block["end"]),
                    "reason": block["reason"],
                }
            )

    scheduled_items.sort(key=lambda item: parse_iso(item["start_iso"]))
    infeasible = [state.memo.id for state in memo_states if (not state.assigned and state.remaining > 0)]

    return {
        "scheduled_items": scheduled_items,
        "debug_log": debug_log,
        "infeasible": infeasible,
        "parameters": asdict(params),
    }



In [ ]:
events, memos = load_sample_day()
result = allocate_day(events, memos)

print("Scheduled blocks:")
for item in result["scheduled_items"]:
    print(f"{item['start_iso']} -> {item['end_iso']} | {item['title']} ({item['memo_id']})")

print("\nInfeasible memos:", result["infeasible"])
print("\nFirst 10 debug entries:")
for entry in result["debug_log"][:10]:
    print(entry)



In [ ]:
scheduled_df = pd.DataFrame(result["scheduled_items"])
if not scheduled_df.empty:
    scheduled_df["start"] = pd.to_datetime(scheduled_df["start_iso"])
    scheduled_df["end"] = pd.to_datetime(scheduled_df["end_iso"])
    scheduled_df["duration_min"] = (scheduled_df["end"] - scheduled_df["start"]).dt.total_seconds() / 60
    display(scheduled_df[["memo_id", "title", "start", "end", "duration_min"]])

    print("\nSummary by memo:")
    summary = scheduled_df.groupby("memo_id")["duration_min"].sum().sort_values(ascending=False)
    display(summary)
else:
    print("No scheduled blocks to show.")



In [ ]:
def build_gap_usage_table(result: Dict[str, Any], events: List[CalendarEvent], memos: List[Memo]) -> pd.DataFrame:
    """Return a per-gap, per-block breakdown showing how open time was consumed."""
    normalized_events = normalize_events(events)
    day_start, day_end = derive_day_bounds(normalized_events, memos)
    gaps = build_gaps(normalized_events, day_start, day_end)
    scheduled_items = sorted(result["scheduled_items"], key=lambda item: item["start_iso"])

    rows: List[Dict[str, Any]] = []
    for gap_index, gap in enumerate(gaps, start=1):
        gap_length = difference_in_minutes(gap.start, gap.end)
        cumulative_used = 0.0
        for item in scheduled_items:
            start = parse_iso(item["start_iso"])
            end = parse_iso(item["end_iso"])
            if end <= gap.start or start >= gap.end:
                continue

            overlap_start = max(start, gap.start)
            overlap_end = min(end, gap.end)
            duration = difference_in_minutes(overlap_start, overlap_end)
            if duration <= 0:
                continue

            cumulative_used += duration
            rows.append(
                {
                    "gap_index": gap_index,
                    "gap_start": gap.start,
                    "gap_end": gap.end,
                    "gap_length_min": gap_length,
                    "memo_id": item["memo_id"],
                    "title": item["title"],
                    "block_start": overlap_start,
                    "block_end": overlap_end,
                    "block_duration_min": duration,
                    "relative_start_min": difference_in_minutes(gap.start, overlap_start),
                    "relative_end_min": difference_in_minutes(gap.start, overlap_end),
                    "cumulative_used_min": cumulative_used,
                }
            )

    return pd.DataFrame(rows)


def plot_gap_consumption(gap_usage: pd.DataFrame):
    """Visualise each gap as a line with expanding dots representing allocated blocks."""
    if gap_usage.empty:
        print("No gap usage to plot.")
        return None

    gap_groups = list(gap_usage.groupby("gap_index", sort=True))
    fig, axes = plt.subplots(len(gap_groups), 1, figsize=(12, 2.8 * len(gap_groups)), sharex=False)
    if len(gap_groups) == 1:
        axes = [axes]

    cmap = plt.get_cmap("tab20")
    for ax, (gap_index, group) in zip(axes, gap_groups):
        sorted_group = group.sort_values("relative_start_min")
        gap_length = sorted_group["gap_length_min"].iloc[0]
        gap_start_label = sorted_group["gap_start"].iloc[0].strftime("%H:%M")
        gap_end_label = sorted_group["gap_end"].iloc[0].strftime("%H:%M")

        ax.hlines(0, 0, gap_length, color="#B0B0B0", linewidth=2, linestyles="dashed")

        for idx, row in enumerate(sorted_group.itertuples()):
            start = row.relative_start_min
            duration = row.block_duration_min
            center = start + duration / 2
            color = cmap(idx % cmap.N)

            ax.plot([start, start + duration], [0, 0], color=color, linewidth=6, solid_capstyle="round")
            ax.scatter(center, 0, s=(max(duration, 1) * 12) ** 1.2, color=color, alpha=0.35, edgecolors="none")
            ax.text(center, 0.18, f"{row.title}\n{duration:.0f} min", ha="center", va="bottom", fontsize=9)
            ax.text(center, -0.22, f"{row.cumulative_used_min:.0f} / {row.gap_length_min:.0f} min", ha="center", va="top", fontsize=8, color="#555555")

        ax.set_xlim(0, gap_length)
        ax.set_ylim(-0.5, 0.6)
        ax.set_yticks([])
        ax.set_xlabel("Minutes from gap start →")
        ax.set_title(f"Gap {gap_index}: {gap_start_label}–{gap_end_label}  (length {gap_length:.0f} min)")

    plt.tight_layout()
    return fig


def plot_gap_snapshots(gap_usage: pd.DataFrame):
    """Show initial, intermediate, and final states for each gap's allocation timeline."""
    if gap_usage.empty:
        print("No gap usage to plot.")
        return []

    snapshots = []
    cmap = plt.get_cmap("tab20")
    for gap_index, group in gap_usage.groupby("gap_index", sort=True):
        sorted_group = group.sort_values("relative_start_min").reset_index(drop=True)
        steps = len(sorted_group)
        n_panels = max(1, steps + 1)  # include initial empty state
        fig, axes = plt.subplots(1, n_panels, figsize=(min(5 + 3 * steps, 18), 3.2), sharey=True)
        if n_panels == 1:
            axes = [axes]

        gap_length = sorted_group["gap_length_min"].iloc[0] if steps else difference_in_minutes(group["gap_start"].iloc[0], group["gap_end"].iloc[0])
        gap_start_label = group["gap_start"].iloc[0].strftime("%H:%M")
        gap_end_label = group["gap_end"].iloc[0].strftime("%H:%M")

        for panel_idx, ax in enumerate(axes):
            ax.hlines(0, 0, gap_length, color="#B0B0B0", linewidth=2, linestyles="dashed")
            upto = panel_idx - 1  # -1 means initial empty state
            if upto >= 0:
                subset = sorted_group.iloc[: upto + 1]
                for idx, row in enumerate(subset.itertuples()):
                    start = row.relative_start_min
                    duration = row.block_duration_min
                    center = start + duration / 2
                    color = cmap(idx % cmap.N)

                    ax.plot([start, start + duration], [0, 0], color=color, linewidth=6, solid_capstyle="round")
                    ax.scatter(center, 0, s=(max(duration, 1) * 12) ** 1.2, color=color, alpha=0.35, edgecolors="none")
                    ax.text(center, 0.16, f"{row.title}\n{duration:.0f} min", ha="center", va="bottom", fontsize=8)
                    ax.text(center, -0.18, f"{row.cumulative_used_min:.0f} / {row.gap_length_min:.0f}", ha="center", va="top", fontsize=7, color="#555555")

            ax.set_xlim(0, gap_length)
            ax.set_ylim(-0.45, 0.55)
            ax.set_yticks([])
            ax.set_xlabel("Minutes →")
            if panel_idx == 0:
                title = "Initial"
            elif panel_idx == n_panels - 1:
                title = f"Final (step {steps})"
            else:
                title = f"Step {panel_idx}: {subset.iloc[-1]['title']}"
            ax.set_title(title, fontsize=10)

        fig.suptitle(f"Gap {gap_index}: {gap_start_label}–{gap_end_label} ({gap_length:.0f} min)", fontsize=12, y=1.05)
        fig.tight_layout()
        snapshots.append(fig)

    return snapshots


def build_chronological_sequence(result: Dict[str, Any], events: List[CalendarEvent], memos: List[Memo]) -> List[Dict[str, Any]]:
    """Compose a unified chronological sequence of fixed events and scheduled memo blocks."""
    normalized_events = normalize_events(events)
    scheduled_items = sorted(result["scheduled_items"], key=lambda item: item["start_iso"])
    memo_lookup = {memo.id: memo for memo in memos}

    entries: List[Dict[str, Any]] = []
    for evt in normalized_events:
        entries.append(
            {
                "type": "event",
                "id": evt.raw.id,
                "title": evt.raw.title or evt.raw.id,
                "start": evt.start,
                "end": evt.end,
                "location": evt.location,
            }
        )

    for item in scheduled_items:
        memo = memo_lookup.get(item["memo_id"])
        entries.append(
            {
                "type": "memo",
                "id": item["id"],
                "title": memo.title if memo else item["title"],
                "start": parse_iso(item["start_iso"]),
                "end": parse_iso(item["end_iso"]),
                "location": memo.location if memo else None,
            }
        )

    entries.sort(key=lambda entry: entry["start"])
    return entries


def plot_chronological_grid(sequence: List[Dict[str, Any]], max_items_per_fig: int = 18):
    """Render a grid where chronological entries are connected by arrows."""
    if not sequence:
        print("No chronological data to display.")
        return []

    figures: List[Any] = []
    palette = {
        "event": "#ff6b6b",
        "memo": "#4d96ff",
    }

    for batch_start in range(0, len(sequence), max_items_per_fig):
        batch = sequence[batch_start : batch_start + max_items_per_fig]
        n = len(batch)
        if n == 0:
            continue

        fig, ax = plt.subplots(figsize=(min(4 + 1.6 * n, 22), 4))
        ax.set_xlim(0, n)
        ax.set_ylim(-0.1, 2.1)

        # Draw grid lines
        for x in range(n + 1):
            ax.axvline(x, color="#DDDDDD", linewidth=0.8, zorder=0)
        ax.axhline(1, color="#DDDDDD", linewidth=0.8, zorder=0)
        ax.axhline(0, color="#DDDDDD", linewidth=0.8, zorder=0)
        ax.axhline(2, color="#DDDDDD", linewidth=0.8, zorder=0)

        ax.set_xticks(np.arange(n) + 0.5)
        ax.set_xticklabels([f"{entry['start'].strftime('%H:%M')}" for entry in batch], rotation=45, ha="right")
        ax.set_yticks([0.5, 1.5])
        ax.set_yticklabels(["Suggestions", "Fixed Events"])

        centers = []
        for idx, entry in enumerate(batch):
            row = 1 if entry["type"] == "event" else 0
            color = palette.get(entry["type"], "#999999")
            rect = Rectangle((idx, row), 1, 1, facecolor=color, alpha=0.25, edgecolor=color, linewidth=1.5)
            ax.add_patch(rect)

            center = (idx + 0.5, row + 0.5)
            centers.append(center)

            label_lines = [entry["title"]]
            duration_min = difference_in_minutes(entry["start"], entry["end"])
            label_lines.append(f"{duration_min:.0f} min")
            if entry["location"]:
                label_lines.append(f"@ {entry['location']['lat']:.3f},{entry['location']['lng']:.3f}")
            ax.text(center[0], center[1], "\n".join(label_lines), ha="center", va="center", fontsize=9)

        for (x0, y0), (x1, y1) in zip(centers[:-1], centers[1:]):
            ax.annotate(
                "",
                xy=(x1, y1),
                xytext=(x0, y0),
                arrowprops=dict(arrowstyle="->", color="#444444", linewidth=1.5),
            )

        total_label = f"Chronological sequence {batch_start + 1}–{batch_start + n} of {len(sequence)}"
        ax.set_title(total_label)
        ax.set_ylabel("Type")
        ax.set_xlabel("Start time (local)")
        ax.set_yticks([0.5, 1.5])
        ax.set_yticklabels(["Suggestions", "Fixed Events"])
        ax.set_ylim(-0.1, 2.1)
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["left"].set_visible(False)
        ax.spines["bottom"].set_visible(False)

        fig.tight_layout()
        figures.append(fig)

    return figures


def render_location_grid(sequence: List[Dict[str, Any]], rows: int = 10, cols: int = 10):
    """Project chronological entries with locations onto a coarse lat/lng grid, connected by arrows."""
    if not sequence:
        print("No chronological data provided.")
        return None

    geo_entries: List[Dict[str, Any]] = []
    for order, entry in enumerate(sequence, start=1):
        location = entry.get("location")
        if not location:
            continue
        duration = difference_in_minutes(entry["start"], entry["end"])
        geo_entries.append(
            {
                "order": order,
                "type": entry["type"],
                "title": entry["title"],
                "start": entry["start"],
                "duration": duration,
                "lat": location["lat"],
                "lng": location["lng"],
            }
        )

    if not geo_entries:
        print("No location data to display.")
        return None

    df = pd.DataFrame(geo_entries)
    lat_min, lat_max = df["lat"].min(), df["lat"].max()
    lng_min, lng_max = df["lng"].min(), df["lng"].max()
    if lat_min == lat_max:
        lat_max += 1e-3
    if lng_min == lng_max:
        lng_max += 1e-3

    def project(value: float, lower: float, upper: float, bins: int) -> int:
        ratio = (value - lower) / (upper - lower)
        return int(round(clamp(ratio, 0, 1) * (bins - 1)))

    palette = {
        "event": "#ff6b6b",
        "memo": "#4d96ff",
    }

    fig, ax = plt.subplots(figsize=(cols * 0.9, rows * 0.9))
    ax.set_xlim(0, cols)
    ax.set_ylim(0, rows)

    ax.set_xticks(np.arange(cols) + 0.5)
    ax.set_yticks(np.arange(rows) + 0.5)
    ax.set_xticklabels([f"{lng:.3f}" for lng in np.linspace(lng_min, lng_max, cols)])
    ax.set_yticklabels([f"{lat:.3f}" for lat in np.linspace(lat_max, lat_min, rows)])

    ax.set_xticks(np.arange(0, cols + 1, 1), minor=True)
    ax.set_yticks(np.arange(0, rows + 1, 1), minor=True)
    ax.grid(which="minor", linestyle="--", linewidth=0.6, color="#B8B8B8")
    ax.tick_params(which="minor", bottom=False, left=False, labelbottom=False, labelleft=False)

    centers: List[Tuple[float, float]] = []
    for entry in geo_entries:
        row_idx = rows - 1 - project(entry["lat"], lat_min, lat_max, rows)
        col_idx = project(entry["lng"], lng_min, lng_max, cols)
        center = (col_idx + 0.5, row_idx + 0.5)
        centers.append(center)

        color = palette.get(entry["type"], "#999999")
        rect = Rectangle((col_idx + 0.05, row_idx + 0.05), 0.9, 0.9, facecolor=color, alpha=0.25, edgecolor=color, linewidth=1.2)
        ax.add_patch(rect)

        label = "\n".join(
            [
                f"#{entry['order']} {entry['title']}",
                f"{entry['start'].strftime('%H:%M')} ({entry['duration']:.0f} min)",
            ]
        )
        ax.text(center[0], center[1], label, ha="center", va="center", fontsize=8, wrap=True)

    for (x0, y0), (x1, y1) in zip(centers[:-1], centers[1:]):
        ax.annotate(
            "",
            xy=(x1, y1),
            xytext=(x0, y0),
            arrowprops=dict(arrowstyle="->", color="#444444", linewidth=1.2),
        )

    ax.set_title("Chronological Location Grid (10×10)")
    ax.invert_yaxis()
    ax.set_aspect("equal")
    plt.tight_layout()
    return fig



In [ ]:
gap_usage_df = build_gap_usage_table(result, events, memos)
if gap_usage_df.empty:
    print("No scheduled items to visualise; allocator returned an empty plan.")
else:
    display(
        gap_usage_df[
            [
                "gap_index",
                "gap_start",
                "gap_end",
                "memo_id",
                "title",
                "block_duration_min",
                "cumulative_used_min",
            ]
        ]
    )

    gap_fig = plot_gap_consumption(gap_usage_df)
    if gap_fig is not None:
        display(gap_fig)

    snapshot_figs = plot_gap_snapshots(gap_usage_df)
    for fig in snapshot_figs:
        display(fig)

sequence = build_chronological_sequence(result, events, memos)
sequence_figs = plot_chronological_grid(sequence, max_items_per_fig=15)
for fig in sequence_figs:
    display(fig)

location_fig = render_location_grid(sequence, rows=10, cols=10)
if location_fig is not None:
    display(location_fig)
